In [1]:
from bs4 import BeautifulSoup
import requests
import sys
import io
import tempfile
import time

#取得股利資訊
def get_stock_dividend(no):
    url ='https://tw.stock.yahoo.com/d/s/dividend_' + str(no) +'.html'
    req = requests.get(url, timeout=5, allow_redirects=True)
    tmpName = tempfile.gettempdir() + '\\t2.html'
    open(tmpName, 'wb').write(req.content)
    results=[]
    with io.open(tmpName,'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr', {'bgcolor':'#FFFFFF'})
        for tr in trs:
            tds = tr.find_all('td')
            result=[]
            for td in tds:
                result.append(float(td.text))
            results.append(result)
    return results

#取得股價
def get_stock_price(no):
    url ='https://tw.stock.yahoo.com/q/ts?s='+str(no)
    req = requests.get(url, timeout=5, allow_redirects=True)
    tmpName = tempfile.gettempdir() + '\\t'+ no + '.html'
    open(tmpName,'wb').write(req.content)
    result=0.0
    with io.open(tmpName,'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr', {'bgcolor':'#ffffff'})
        if len(trs) > 0:
            tds = trs[0].find_all('td')
            if len(tds) > 3 :
                result= float(tds[3].text)
            else:
                sys.stdout.write('v')
        else:
            sys.stdout.write('z')
    return result;

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import sys

def auth_gss_client(path, scopes):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(path, scopes)
    return gspread.authorize(credentials)

def get_sheet_value(sheet, row, col):
    time.sleep(1.1)
    return sheet.cell(row,col).value

def set_sheet_value(sheet, row, col, value):
    time.sleep(1.1)
    sheet.update_cell(row,col,value)

def set_sheet_values(sheet, row, col, values):
    alpha='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    preA=alpha[col-1]+str(row)
    m=col+len(values)-2
    j=m/len(alpha)
    k=m%len(alpha)
    if j>0:
        preB=alpha[j-1]+alpha[k]+str(row)
    else:
        preB=alpha[col+len(values)-2]+str(row)
    label=preA+':'+preB
    cell_list=sheet.range(label)
    for c,v in zip(cell_list, values):
        c.value = v
    time.sleep(1.1)
    sheet.update_cells(cell_list)

def update_divident(no,rows):
    results = get_stock_dividend(no)
    colIndex = 14
    cell_value = []
    for result in results:
        cell_value.append(result[1])
        cell_value.append(result[2])
        
    tmpVal = get_sheet_value(wks.sheet1, rows ,colIndex)
    if((len(cell_value) > 0) and (tmpVal != cell_value[0])):
        set_sheet_values(wks.sheet1,rows,colIndex, cell_value)
    else:
        sys.stdout.write('x')

auth_json_path = 'auth.json'
gss_scopes = ['https://spreadsheets.google.com/feeds']

gss_client = auth_gss_client(auth_json_path, gss_scopes)
s_key=open('spreadsheet_key','r').read()

wks = gss_client.open_by_key(s_key)
stockNo = 1
lineIndex = 2
run = True
print('Get Price\n')
while run:
    stockNo = get_sheet_value(wks.sheet1, lineIndex ,1)
    if stockNo != '':
        sys.stdout.write(str(stockNo))
        set_sheet_value(wks.sheet1, lineIndex, 3, get_stock_price(stockNo))
        divValue = get_sheet_value(wks.sheet1, lineIndex, 14)
        if divValue != '':
            update_divident(stockNo, lineIndex)
            
        sys.stdout.write(',')
        time.sleep(1)
        lineIndex += 1
    else:
        print('Bye!\n')
        run = False

Get Price

1101,1210,1215,1216,1227,1229,1231,1232,1234,1307,1513,1712,1717,1773,2105,2108,2301,2317,2324,2330,2347,2412,2596,2616,2701,2801,2812,2820,2834,2838,2845,2880,2881,2882,2883,2884,2885,2886,2887,2888,2889,2890,2891,2892,2905,2912,3028,3045,3702,4711,4904,5438,5519,5880,6136,6151,6183,6189,6216,6218,6281,6803,8422,8926,9905,9927,9939,5312,2535,5410,1904,1907,5522,8905,2065,2546,5511,6186,2542,Bye!



In [3]:
from bs4 import BeautifulSoup
import requests
import io
import tempfile

def get_stock_priceDebug(no):
    tmpName = tempfile.gettempdir() + '\\t'+ no + '.html'
    result=0.0
    with io.open(tmpName,'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr', {'bgcolor':'#ffffff'})
        tds = trs[0].find_all('td')
        print(trs)
        if len(tds) > 3 :
            result= float(tds[3].text)
    return result;

print(get_stock_priceDebug('2317'))

[<tr align="center" bgcolor="#ffffff" height="25"><td>13:30:00</td><td>77.40</td><td class="high">77.50</td><td>77.40</td><td>\uff0d</td><td>4401</td></tr>, <tr align="center" bgcolor="#ffffff" height="25"><td>13:24:59</td><td class="high">77.60</td><td class="high">77.70</td><td class="high">77.70</td><td class="high">\u25b30.3</td><td>1</td></tr>, <tr align="center" bgcolor="#ffffff" height="25"><td>13:24:54</td><td class="high">77.60</td><td class="high">77.70</td><td class="high">77.70</td><td class="high">\u25b30.3</td><td>1</td></tr>, <tr align="center" bgcolor="#ffffff" height="25"><td>13:24:49</td><td class="high">77.60</td><td class="high">77.70</td><td class="high">77.70</td><td class="high">\u25b30.3</td><td>1</td></tr>, <tr align="center" bgcolor="#ffffff" height="25"><td>13:24:44</td><td class="high">77.60</td><td class="high">77.70</td><td class="high">77.70</td><td class="high">\u25b30.3</td><td>2</td></tr>, <tr align="center" bgcolor="#ffffff" height="25"><td>13:24:39</

sheetfu

In [16]:
from sheetfu import SpreadsheetApp

spreadsheet_key = open('spreadsheet_dbg_key','r').read()
spreadsheet = SpreadsheetApp('auth.json').open_by_id(spreadsheet_key)
sheet = spreadsheet.get_sheet_by_name('Sheet1')
data_range = sheet.get_data_range()

values = data_range.get_values()
#backgrounds = data_range.get_backgrounds()

print(data_range)
print(values)

<Range object A1:I3>
[[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [666, 2, 3, 4, 5, 6, 7, 8, 9]]
